This file contains the code to find the most suitable K value for the KNN filling algorithm
Since Logistic regression performs and worst among all algorithm, we don't use it here

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
# Cross Validation Classification Accuracy
from pandas import read_csv
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
from sklearn import preprocessing 
from sklearn.impute import KNNImputer
# from sklearn.neighbors import KNeighborsClassifier
#import faiss
from numpy import isnan
#import matplotlib.pyplot as plot
from matplotlib import pyplot
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score
from sklearn import tree
import statistics
import time
import warnings
warnings.filterwarnings('ignore')
# plotting errorbar graph for visual representation of the performance of the classifiers
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
import random
from xgboost import XGBClassifier
import shutil
import os
from os import path
import ipynb.fs.full.our_functions_library as flib
from our_evaluate_sepsis_score import evaluate_performance
#from our_functions_library import *

In [2]:
# Global initialization, get all the data from file and generate all the needed variable
filename = 'raw_data/raw_data_2000.csv' # use raw dataset
#filename = 'data.csv' # use raw dataset
originalData = read_csv(filename) # read csv data into DataFrame var raw
print("Data size:",originalData.shape)
Original_Uniq_ID= np.unique(originalData['Patient_id']) 
Uniq_ID = Original_Uniq_ID.copy()
print('Patient id size:',len(Uniq_ID))
X_columns = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 'pH', 
             'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
             'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 
             'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2','HospAdmTime',
             'ICULOS', 'Patient_id', 'time']
y_columns = ['Patient_id', 'SepsisLabel']

# To test if these 3 columns is useful
# originalData = originalData.drop(['Unit1', 'Unit2', 'EtCO2'], axis=1)
# X_columns.remove('Unit1')
# X_columns.remove( 'Unit2')
# X_columns.remove( 'EtCO2')

# Initialize the empty array X_train, X_test, y_train, y_test
X_train = pd.DataFrame(columns = X_columns)
X_test = pd.DataFrame(columns = X_columns)
y_train = pd.DataFrame(columns = y_columns)
y_test = pd.DataFrame(columns = y_columns)
# Below are the lists for KNN results

KNN_UtilityScore_mean = []
KNN_UtilityScore_std = []

KNN_F1Score_mean = []
KNN_F1Score_std = []

KNN_auroc_mean = []
KNN_auprc_mean = []

KNN_accuracy_mean = []
KNN_accuracy_std = []

KNN_positiveprediction_mean = []
KNN_baseline_mean = [ ]

KNN_total_Time= [ ]


fillmethod =""  

Data size: (77726, 43)
Patient id size: 2000


In [3]:
#This block will shuffle the id sets of patients in a fix manner, so for every time you run the code, the dataset and trainset are always the same.
seed = 2
random.seed(seed)
if(np.array_equal(Original_Uniq_ID,Uniq_ID) ):
    print("They are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.")   
    random.shuffle(Uniq_ID)# randomly sorted the patient IDs    
else:
    print("They are not the same. Uniq_id has already been shuffled.")    
print("The original uniq id set is:\n",Original_Uniq_ID[1:20])
print("Uniq id:************************************")
print("The fixed shuffelld id set, it should be 211 275 153 189 184 110 124  49 345...\n",Uniq_ID[1:20])

They are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.
The original uniq id set is:
 [ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
Uniq id:************************************
The fixed shuffelld id set, it should be 211 275 153 189 184 110 124  49 345...
 [1530  469  774  749 1165 1780 1525  922 1280  509 1689  298  616 1551
 1840  327 1865  752 1031]


In [4]:
# #fill the missing data with one function
# print(filename)
# #print(originalData)
# if (originalData.isnull().values.any()):
#     print('There is data missing in the original data set')
# dataByPatient = originalData.groupby('Patient_id') # data grouped by Patient_id

In [5]:
def clearAllDatasets():
    global X_train, X_test, y_train, y_test
    if not X_train.empty: 
        X_train = X_train[0:0]
    if not X_test.empty:
        X_test = X_test[0:0]
    if not y_train.empty:
        y_train = y_train[0:0]
    if not y_test.empty:
        y_test = y_test[0:0]
        
def KNN_reset():
    #KNN with different K size, 
    KNN_UtilityScore_mean.clear()
    KNN_UtilityScore_std.clear()
    KNN_F1Score_mean.clear()
    KNN_F1Score_std.clear()
    KNN_auroc_mean.clear()
    KNN_auprc_mean.clear()
    KNN_accuracy_mean.clear()
    KNN_accuracy_std.clear()
    KNN_positiveprediction_mean.clear()
    KNN_baseline_mean.clear()
    KNN_total_Time.clear()
    
def generateTrainDataSet(patienIds):
    global X_train, y_train
    for i in patienIds:
#     print('Patient_id',i,':\n',dataByPatient.get_group(i),'\n')
        X_train = X_train.append(dataByPatient.get_group(i).loc[:, X_columns])
        y_train = y_train.append(dataByPatient.get_group(i).loc[:, y_columns])
    return X_train, y_train
 
def generateTestDataSet(patienIds):
    global X_test, y_test
    for i in patienIds:
#     print('Patient_id',i,':\n',dataByPatient.get_group(i),'\n')
        X_test = X_test.append(dataByPatient.get_group(i).loc[:, X_columns])
        y_test = y_test.append(dataByPatient.get_group(i).loc[:, y_columns]) 
    return X_test, y_test

# def CalculateSOFAScore(X_train_impute, X_test_impute):
#     pass

In [6]:
# Code for K-fold algorithm, Before this step the missing data has been filled, 
def KFold_patient(model, KforKFold=10,KforKNN=5, fillmethod=""):
    start = time.time()# time indicator for how long the Kfold func takes
    global X_train, X_test, y_train, y_test
    print("K Fold of ",KforKFold ," folds with KNN =",KforKNN)
    
    # initialisation of the array for storing the different intermediate results
    accuracy_model = []
    F1Score_model = []
    baseline_model= []
    auroc_model = []
    auprc_model = []
    physio_accuracy_model = []
    f_measure_model = []
    utility_score_model = []
    mean_train = 0
    positivepredictions = []
    #The unique id sets have been created and shuffled in a fix manner in the third block, you can just use it here and no more any other manipulation
    idSets = np.array_split(Uniq_ID, KforKFold)# divide the ids into K groups
    #print("Patients number: ", len(Uniq_ID)) #print('idSets arrays',idSets)
    #This for loop is for Kfold, calculating the results for K times
    for i in range(KforKFold):
        start1 = time.time()
        clearAllDatasets()
#         X_train, X_test, y_train, y_test = flib.clearAllDatasets(X_train, X_test, y_train, y_test) #first clear all the datasets
        print("for the",i+1,"th iteration",idSets[i])   
        t1 = time.time()
        X_test,y_test = generateTestDataSet(idSets[i])
        print("Time for splitting id to test:", round(time.time()- t1,2))
        
        t2= time.time()
        for j in range(KforKFold):
            if j != i:
                 X_train, y_train = generateTrainDataSet(idSets[j])
        print("Time for splitting id to train:", round(time.time()- t2,2))

        #printDataset()
        #Now the train and test dataset is generated
        #we can begin to train the model wit the training set and evaaulate the performance with the test sett   
#         X_train = X_train.drop('time', 1) X_test = X_test.drop('time', 1)
        X_train=X_train.astype('float64')
        X_test=X_test.astype('float64')    
        YTest_copy = y_test  # variable of joining the filled data (X) and Y (Train_output) 
        patientID_ytest = y_test['Patient_id']
        y_train = y_train.drop('Patient_id', 1)
        y_test = y_test.drop('Patient_id', 1)      
        y_train=y_train.astype('float64')
        y_test=y_test.astype('float64')
        #print('YTest',YTest_copy.head())
        
        #fill the missing data
        if X_train.isnull().values.any() or X_test.isnull().values.any() :
            print("X_train or X_test contains NaN values, KNN/mean is performed.")
            #if there is missing value
            X_train_impute, X_test_impute, fillmethod = flib.KNNfilling(X_train, X_test, KforKNN, fillmethod)
            #X_train_impute, X_test_impute,fillmethod = flib.MeanFilling(X_train,X_test, fillmethod)
            #check the missing data   
            if np.isnan(X_train_impute).any() or np.isnan(X_test_impute).any() :
                print("X_train_impute or X_test_impute still contains NaN values")        
            else:
                print("X_train_impute or X_test_impute have all been filled ")  
        else:
            print("X_train or X_test have all been filled ")
            X_train_impute = X_train
            X_test_impute = X_test
                 
        #Scale the data， uncomment this part for logistic regression only, in other case, put them in comment
        model_name = type(model).__name__
        if (model_name == 'LogisticRegression'):
            print("scaled!")
            scaler = preprocessing.StandardScaler()
            scaler.fit(X_train_impute)
            X_train_impute = scaler.transform(X_train_impute)
            X_test_impute = scaler.transform(X_test_impute)

        #fit the model and predict
        model.fit(X_train_impute, y_train)
        y_predicted = model.predict(X_test_impute)   
        y_predicted_probobility = model.predict_proba(X_test_impute) 
        #transfer the outpu and evalute it
        y_labels = y_test.astype(int).to_numpy()
        y_predicted = y_predicted.astype(int)
        y_predicted_probobility =  y_predicted_probobility[:,1].round(4)
        #print(y_labels)      
        auroc, auprc, physio_accuracy, f_measure, utility_score = evaluate_performance(y_labels, y_predicted, y_predicted_probobility,patientID_ytest)
#         print("\nauroc",round(auroc,4),"auprc",round(auprc,4),"util_accuracy",round(accuracy1,4))
#         print("f_measure",round(f_measure,4),"utility_score",round(utility_score,4))  
        auroc_model.append(round(auroc,4))
        auprc_model.append(round(auprc,4))
        physio_accuracy_model.append(round(physio_accuracy,4))
        f_measure_model.append(round(f_measure,4))
        utility_score_model.append(round(utility_score,4))
        positivepredictions.append(np.sum(y_predicted))

        num_rows= X_test_impute.shape[0]
        scores = np.zeros(num_rows).astype('float64')# score parameter:output from get_sepsis_score
        labels = np.zeros(num_rows).astype('float64')
        #WriteToFiles(X_train_impute,YTest_copy,i, fillmethod)
        #CalcMean_Std(X_train_impute)
        #take down the results
        accuracy_model.append((accuracy_score(y_test, y_predicted, normalize=True)*100).round(2))
        F1Score_model.append((f1_score(y_test, y_predicted)*100).round(2))
        baseline_model.append(round( (1 - float(y_test.mean()) )*100 , 2 ))
        print("\ny_test size:",y_test.shape, '1´s in y_test',y_test.sum())
        #baseline_model.append(((1-y_test.mean())*100).round(2))
        print("The number of 1 (SepsisLabel) in this prediction: ", np.sum(y_predicted))
        end1 = time.time()
        print("Time spent in this KFold iteration",round(end1-start1,2),"sec.\n")
        print("******************************************************************")
    print('Accuracy model:', accuracy_model)
    print('F1_score model:', F1Score_model)
    print('Baseline model:', baseline_model)
    
    print("\nEvaluation parameters of the utiltiy evaluation function:")
    print('auroc of model:', auroc_model)
    print('auprc of model:', auprc_model)
    print('Utility accuracy of model:', accuracy_model)
    print('utility F1 of model:', f_measure_model)
    print('Utility score of model:', utility_score_model)
    
    KNN_UtilityScore_mean.append(np.mean(utility_score_model))
    KNN_UtilityScore_std.append(np.std(utility_score_model))
    
    KNN_F1Score_mean.append(np.mean(f_measure_model))
    KNN_F1Score_std.append(np.std(f_measure_model))
    
    KNN_auroc_mean.append(np.mean(auroc_model))
    KNN_auprc_mean.append(np.mean(auprc_model))
    
    KNN_accuracy_mean.append(np.mean(physio_accuracy_model))
    KNN_accuracy_std.append(np.std(physio_accuracy_model))
    
    KNN_positiveprediction_mean.append(np.mean(positivepredictions))
    KNN_baseline_mean.append(np.mean(baseline_model))
    
    totalTime=round(time.time()- start,2)
    KNN_total_Time.append(totalTime)
    
    print("\nTotal Time spent in  KFold function",totalTime,"sec.\n")


In [7]:
def displayCurrentResult(KforKNNstart,KforKNNend):
    print("KNN_UtilityScore_mean",KNN_UtilityScore_mean)
    print("KNN_UtilityScore_std",KNN_UtilityScore_std)
    print("KNN_F1Score_mean",KNN_F1Score_mean)
    print("KNN_F1Score_std",KNN_F1Score_std)
    print("KNN_auroc_mean",KNN_auroc_mean)
    print("KNN_auprc_mean",KNN_auprc_mean)
    print("KNN_accuracy_mean",KNN_accuracy_mean)
    print("KNN_accuracy_std",KNN_accuracy_std)
    print("KNN_positiveprediction_mean",KNN_positiveprediction_mean)
    print("KNN_baseline_mean",KNN_baseline_mean)        
    print(len(KNN_accuracy_mean))
    print(len(KNN_accuracy_std))
    print(len(KNN_F1Score_mean))
    print(len(KNN_F1Score_std))
    print(len(KNN_positiveprediction_mean))
    plotKNNResultFigure(KforKNNstart,KforKNNend,KNN_UtilityScore_mean,"Mean Utility Score vs K",xlabel='K',ylabel="mean Utility Score")
    plotKNNResultFigure(KforKNNstart,KforKNNend,KNN_UtilityScore_std,"std Utility Score vs K",xlabel='K',ylabel="std Utility Score")
    plotKNNResultFigure(KforKNNstart,KforKNNend,KNN_F1Score_mean,"Mean F1 score vs K",xlabel='K',ylabel="Mean F1 score")
    plotKNNResultFigure(KforKNNstart,KforKNNend,KNN_F1Score_std,"Std F1 score vs K",xlabel='K',ylabel="Std F1 score")
    plotKNNResultFigure(KforKNNstart,KforKNNend,KNN_accuracy_mean,"Mean Accuracy vs K",xlabel='K',ylabel="Mean Accuracy")      
def findBestKforKNN(model, KforKFold=10,KforKNNstart=1,KforKNNend=10, stepsize=5):
    if(KforKNNend<=1):
        print("K must be a interger larger than 1")
        return
    KNN_reset()
    step=stepsize
    for i in range(KforKNNstart,KforKNNend+1,step) : 
        print("KNN of K = ",i)    
        KFold_patient(model,10,i)
    print("Now all the training is finished.")

In [8]:
#Here is how to train the model
#logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='saga', max_iter=1000,penalty='l1')
logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='lbfgs', max_iter=1000)
decisionTreeModel = tree.DecisionTreeClassifier(random_state=2)
randomForestModel = RandomForestClassifier(random_state=2)
XGBoostModel = XGBClassifier(random_state=2)
#AdaBoostModel = AdaBoostClassifier(random_state=2)

In [9]:
#fill the missing data with one function
print(filename)
#print(originalData)
if (originalData.isnull().values.any()):
    print('Data isn´t all filled before K-Fold Func')
    data=originalData
#data,fillmethod = flib.linearFillingAll(originalData,1, True)#forwardfilling + fill other missing data with 0
dataByPatient = data.groupby('Patient_id') # data grouped by Patient_id

raw_data/raw_data_2000.csv
Data isn´t all filled before K-Fold Func


In [10]:
model_name= decisionTreeModel.__class__.__name__
if (model_name == 'DecisionTreeClassifier'):
    print("oke")

oke


In [11]:
# print("Logistic Regression")
#KFold_patient(decisionTreeModel,10, 1)

In [12]:
# print("Decision tree")
#KFold_patient(randomForestModel,10, 2)

In [13]:
# findBestKforKNN(logisticRegressionModel,10,1,10)
# findBestKforKNN(XGBoostModel,10,110,120)

In [ ]:
findBestKforKNN(decisionTreeModel,10,1,5)
os.system('\a')

KNN of K =  1
K Fold of  10  folds with KNN = 1
for the 1 th iteration [1367 1530  469  774  749 1165 1780 1525  922 1280  509 1689  298  616
 1551 1840  327 1865  752 1031 1089  714  291   68 1735  404 1285 1389
 1971 1582 1609 1217  499 1120  324  544 1994 1346  501  651 1586  927
  778  503 1450 1566 2000 1835 1194  656  387 1009  704 1248 1429 1424
  745  266  429  542 1932 1361 1011  839 1220 1554  567 1252  281  196
 1413 1386 1416  128  176  875  724  638  479  259  793 1934  817  869
 1898  211 1691  306 1235  670  285  853  552 1434 1576  368  162  314
  418 1322 1316 1170  696 1921 1976  573 1763 1123 1333  729 1845 1634
  267 1863 1095  108  379 1453  312 1229 1830 1886  708 1232 1082 1121
 1606 1068  840 1874  375 1883 1838  748 1318  975 1069  871 1732   87
 1500 1523 1022 1837  134  182 1514 1065 1967   39 1510 1019 1214  543
 1710 1336  821 1038 1908 1399  895  944  398  153 1193  384  599 1086
  199  156 1457  245 1795 1478 1144  717 1066  661  352  549  984  340
  168 

In [ ]:
displayCurrentResult(1,9)